In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from config import mysqlpassword 

In [2]:
players_csv = "Resources/players.csv"
players_df = pd.read_csv(players_csv, index_col = 0)
players_df.head()

,Player,height,weight,collage,born,birth_city,birth_state
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky


In [3]:
# Create key
players_df['namekey'] = players_df['Player'].str.lower()
for s in [" ", "'", "-", ".", ",", "*"]:
    players_df['namekey'] = players_df['namekey'].str.replace(s,'')

# players_df.set_index('namekey', verify_integrity=True, inplace=True)

players_df.head()

,Player,height,weight,collage,born,birth_city,birth_state,namekey
0,Curly Armstrong,180.0,77.0,Indiana University,1918.0,NaN,NaN,curlyarmstrong
1,Cliff Barker,188.0,83.0,University of Kentucky,1921.0,Yorktown,Indiana,cliffbarker
2,Leo Barnhorst,193.0,86.0,University of Notre Dame,1924.0,NaN,NaN,leobarnhorst
3,Ed Bartels,196.0,88.0,North Carolina State University,1925.0,NaN,NaN,edbartels
4,Ralph Beard,178.0,79.0,University of Kentucky,1927.0,Hardinsburg,Kentucky,ralphbeard


In [4]:
players_df.describe()

,height,weight,born
count,3921.000000,3921.000000,3921.00000
mean,198.704922,94.783219,1962.37975
std,9.269761,12.039515,20.33491
min,160.000000,60.000000,1913.00000
25%,190.000000,86.000000,1948.00000
50%,198.000000,95.000000,1964.00000
75%,206.000000,102.000000,1979.00000
max,231.000000,163.000000,1997.00000


In [5]:
# Drop duplicates
players_df.drop_duplicates(subset=['namekey', 'height', 'weight', 'collage', 'born', 'birth_city', 'birth_state'], inplace=True)
players_df.describe()

,height,weight,born
count,3919.000000,3919.000000,3919.000000
mean,198.702730,94.783363,1962.378413
std,9.268865,12.038723,20.339900
min,160.000000,60.000000,1913.000000
25%,190.000000,86.000000,1948.000000
50%,198.000000,95.000000,1964.000000
75%,206.000000,102.000000,1979.000000
max,231.000000,163.000000,1997.000000


In [6]:
players_df.dropna(how='all', inplace=True)
players_df = players_df[['namekey', 'Player', 'height', 'weight', 'collage', 'birth_city', 'birth_state']]
players_df.head()

,namekey,Player,height,weight,collage,birth_city,birth_state
0,curlyarmstrong,Curly Armstrong,180.0,77.0,Indiana University,NaN,NaN
1,cliffbarker,Cliff Barker,188.0,83.0,University of Kentucky,Yorktown,Indiana
2,leobarnhorst,Leo Barnhorst,193.0,86.0,University of Notre Dame,NaN,NaN
3,edbartels,Ed Bartels,196.0,88.0,North Carolina State University,NaN,NaN
4,ralphbeard,Ralph Beard,178.0,79.0,University of Kentucky,Hardinsburg,Kentucky


In [7]:
players_df = players_df.rename(columns= {"collage": "college"})
players_df.head()

,namekey,Player,height,weight,college,birth_city,birth_state
0,curlyarmstrong,Curly Armstrong,180.0,77.0,Indiana University,NaN,NaN
1,cliffbarker,Cliff Barker,188.0,83.0,University of Kentucky,Yorktown,Indiana
2,leobarnhorst,Leo Barnhorst,193.0,86.0,University of Notre Dame,NaN,NaN
3,edbartels,Ed Bartels,196.0,88.0,North Carolina State University,NaN,NaN
4,ralphbeard,Ralph Beard,178.0,79.0,University of Kentucky,Hardinsburg,Kentucky


In [8]:
# Create Database Connection
rds_connection_string = f"root:{mysqlpassword}@localhost/basketball_db"
engine = create_engine(f'mysql://{rds_connection_string}')

# Confirm tables
engine.table_names()

['player_data', 'players', 'salary']

In [9]:
# Load Dataframe into database
players_df.to_sql(name='players', con=engine, if_exists='append', index=False)

In [10]:
# Confirm data added succesfully
pd.read_sql_query('select * from players', con=engine).head()

,namekey,Player,height,weight,college,birth_city,birth_state
0,aaronbrooks,Aaron Brooks,183.0,73.0,University of Oregon,Seattle,Washington
1,aarongordon,Aaron Gordon,206.0,99.0,University of Arizona,San Jose,California
2,aarongray,Aaron Gray,213.0,122.0,University of Pittsburgh,Tarzana,California
3,aaronharrison,Aaron Harrison,198.0,95.0,University of Kentucky,San Antonio,Texas
4,aaronjames,Aaron James,203.0,95.0,Grambling State University,New Orleans,Louisiana
